In [1]:
import plotly.graph_objects as go
import numpy as np
from numpy import cos, sin
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, clear_output

## Calculations 

In [2]:
TABLE_RADIUS = 3
TABLE_ANGLE = np.deg2rad(80)
BASE_RADIUS = 7
BASE_ANGLE = np.deg2rad(15)
MAX_POSSIBLE_HEIGHT = 15
MIN_POSSIBLE_HEIGHT = 0
ARM_LENGTH = 8

def populate_transformation_matrix(x, y, z, phi, theta, psi):
    transformation_matrix = np.array([
        [cos(psi)*cos(theta), cos(psi)*sin(theta)*sin(phi)-sin(psi)*cos(phi), cos(psi)*sin(theta)*cos(phi)+sin(psi)*sin(phi), x],
        [sin(psi)*cos(theta), sin(psi)*sin(theta)*sin(phi)+cos(psi)*cos(phi), sin(psi)*sin(theta)*cos(phi)-cos(psi)*sin(phi), y],
        [-sin(theta), cos(theta)*sin(phi), cos(theta)*cos(phi), z],
        [0, 0, 0, 1]
    ])
    return transformation_matrix

def populate_hexagon(radius, angle_in_radians):
    points = np.array([
        [radius * np.sin(np.deg2rad(60)*0+angle_in_radians/2), radius * np.cos(np.deg2rad(60)*0+angle_in_radians/2), 0],
        [radius * np.sin(np.deg2rad(60)*0-angle_in_radians/2), radius * np.cos(np.deg2rad(60)*0-angle_in_radians/2), 0],
        
        [radius * np.sin(np.deg2rad(60)*2+angle_in_radians/2), radius * np.cos(np.deg2rad(60)*2+angle_in_radians/2), 0],
        [radius * np.sin(np.deg2rad(60)*2-angle_in_radians/2), radius * np.cos(np.deg2rad(60)*2-angle_in_radians/2), 0],
        
        [radius * np.sin(np.deg2rad(60)*4+angle_in_radians/2), radius * np.cos(np.deg2rad(60)*4+angle_in_radians/2), 0],
        [radius * np.sin(np.deg2rad(60)*4-angle_in_radians/2), radius * np.cos(np.deg2rad(60)*4-angle_in_radians/2), 0],
    ])
    return points

def populate_centroid(points):
    centroid = np.mean(points, axis=0)
    points_with_centroid = np.append(points, [centroid], axis=0)
    return points_with_centroid

def transform_point(point, transformation_matrix):
    point_copy = np.array(point) 
    return np.append(point_copy, 1).dot(transformation_matrix.T)[:3]

def find_intersection(table_transformed_point, base_point):
    # We calculate intersection points of a vertical line with a sphere. 
    # Sphere equation: (x-table_transformed_point_x)**2 + (y-table_transformed_point_y)**2 + (z-table_transformed_point_x)**2 = ARM_LENGTH**2
    # Vertical line equation: x = base_point_x; y = base_point_y; z is changing.
    table_transformed_point_x, table_transformed_point_y, table_transformed_point_z = table_transformed_point
    base_point_x, base_point_y, _ = base_point
    
    z_part = ARM_LENGTH**2 - (base_point_x - table_transformed_point_x)**2 - (base_point_y - table_transformed_point_y)**2
    if z_part > 0:
        z_coordinate_of_slider = table_transformed_point_z + np.sqrt(z_part) # One of two solutions
        if MAX_POSSIBLE_HEIGHT > z_coordinate_of_slider > MIN_POSSIBLE_HEIGHT:
            return (base_point_x, base_point_y, z_coordinate_of_slider)
    return None

In [3]:
initial_table_points = populate_centroid(populate_hexagon(TABLE_RADIUS, TABLE_ANGLE))
base_points = populate_hexagon(BASE_RADIUS, BASE_ANGLE)

## Plots and visualisation

In [4]:
AXIS_RANGE = [-15, 15] 
MESH_INDICES = [(0, 1, 6), (2, 3, 6), (4, 5, 6), (1, 4, 6), (3, 0, 6), (5, 2, 6)]
STEP_SIZE = 0.0000001

x_widget = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='X Position')
y_widget = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='Y Position')
z_widget = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='Z Position')
phi_widget = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Phi Angle')
theta_widget = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Theta Angle')
psi_widget = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Psi Angle')
text_widget = widgets.Textarea(
    value='',
    placeholder='Texbox for intersections',
    disabled=False,
    layout={'width': '320px', 'height': '200px'}
)

WIDGETS = [x_widget, y_widget, z_widget, phi_widget, theta_widget, psi_widget]

def find_element_by_tag(fig, tag):
    for i, plot_element in enumerate(fig.data):
        if plot_element.customdata and plot_element.customdata[0] == tag:
            return i
    return None

def format_intersection_points(intersection_points):
    all_positions_found = all(point is not None for point in intersection_points)

    if all_positions_found:
        formatted_intersections = ["Targeted table coordinates can be realized using these slider positions:\n\n"]
        for i, point in enumerate(intersection_points):
            formatted_intersections.append(f"{i}: ({point[0]:.6f}, {point[1]:.6f}, {point[2]:.6f})")
    else:
        formatted_intersections = ["Failed to find slider positions that can realize targeted table coordinates:\n"]
        for i, point in enumerate(intersection_points):
            status = "ok" if point is not None else "None"
            formatted_intersections.append(f"{i}: {status}")

    return '\n'.join(formatted_intersections)


def initialize_plot():
    fig = go.FigureWidget()
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=AXIS_RANGE),
            yaxis=dict(range=AXIS_RANGE),
            zaxis=dict(range=AXIS_RANGE)
        ),
    title='6-DOF robot with vertical parallel rails')
    fig.add_scatter3d(x=initial_table_points[:, 0], y=initial_table_points[:, 1], z=initial_table_points[:, 2], mode='markers', marker=dict(size=5), line=dict(color='blue'), name='Initial Table', customdata=["original_table"])
    fig.add_scatter3d(x=initial_table_points[:, 0], y=initial_table_points[:, 1], z=initial_table_points[:, 2], mode='markers', marker=dict(size=5), line=dict(color='red'), name='Transformed Table', customdata=["transformed_table"])

    for point in base_points:
        fig.add_scatter3d(x=[point[0], point[0]], y=[point[1], point[1]], z=[MIN_POSSIBLE_HEIGHT, MAX_POSSIBLE_HEIGHT], mode='lines', line=dict(color='green', width=2), showlegend=False)
    
    fig.add_mesh3d(
        x=initial_table_points[:, 0], 
        y=initial_table_points[:, 1], 
        z=initial_table_points[:, 2], 
        i=[i[0] for i in MESH_INDICES], 
        j=[i[1] for i in MESH_INDICES], 
        k=[i[2] for i in MESH_INDICES], 
        color='red', 
        opacity=0.5,
        customdata=["table_mesh"]
    )

    for i, (t, b) in enumerate(zip(initial_table_points, base_points)):
        intersection = find_intersection(t, b)
        if intersection:
            _, _, intersection_z = intersection
            fig.add_scatter3d(x=[t[0], b[0]], y=[t[1], b[1]], z=[t[2], intersection_z], mode='lines', line=dict(color='black', width=2), showlegend=False, customdata=[f"arm_{i}"])

    fig.update_layout(scene=dict(xaxis=dict(range= AXIS_RANGE, title='X Axis'), yaxis=dict(range= AXIS_RANGE, title='Y Axis'), zaxis=dict(range= AXIS_RANGE, title='Z Axis'), aspectmode='cube'), margin=dict(l=0, r=0, b=0, t=0))
    return fig

def update_plot(change):
    with plot.batch_update():
        widget_values = [widget.value for widget in WIDGETS]
        transformation_matrix = populate_transformation_matrix(*widget_values)
        transformed_table_points = np.array([transform_point(point, transformation_matrix) for point in initial_table_points])

        transformed_table = find_element_by_tag(plot, "transformed_table")
        mesh = find_element_by_tag(plot, "table_mesh")
        plot.data[transformed_table].x, plot.data[transformed_table].y, plot.data[transformed_table].z = transformed_table_points.T
        plot.data[mesh].x, plot.data[mesh].y, plot.data[mesh].z = transformed_table_points.T
        plot.data[mesh].i, plot.data[mesh].j, plot.data[mesh].k = zip(*MESH_INDICES)

        intersection_points = []
        for i, (t, b) in enumerate(zip(transformed_table_points, base_points)):
            intersection = find_intersection(t, b)
            arm = find_element_by_tag(plot, f"arm_{i}")

            if intersection:
                intersection_points.append(intersection)
                _, _, intersection_z = intersection
                plot.data[arm].x = [t[0], b[0]]
                plot.data[arm].y = [t[1], b[1]]
                plot.data[arm].z = [t[2], intersection_z]
            else:
                intersection_points.append(None)
                plot.data[arm].x = [None, None]
                plot.data[arm].y = [None, None]
                plot.data[arm].z = [None, None]

        intersection_text = format_intersection_points(intersection_points)
        text_widget.value = intersection_text

In [5]:
plot = initialize_plot()
for widget in WIDGETS:
    widget.observe(update_plot, names='value')

In [6]:
x_widget.value = 0.5
y_widget.value = -0.5
z_widget.value = 5
phi_widget.value = -np.pi/10
theta_widget.value = np.pi/3
psi_widget.value = 0

In [7]:
display(plot)

FigureWidget({
    'data': [{'customdata': [original_table],
              'line': {'color': 'blue'},
              'marker': {'size': 5},
              'mode': 'markers',
              'name': 'Initial Table',
              'type': 'scatter3d',
              'uid': '9f2b7672-bb75-4acd-9c76-ad15c9d555db',
              'x': array([ 1.92836283e+00, -1.92836283e+00,  1.02606043e+00,  2.95442326e+00,
                          -2.95442326e+00, -1.02606043e+00,  2.22044605e-16]),
              'y': array([ 2.29813333e+00,  2.29813333e+00, -2.81907786e+00,  5.20944533e-01,
                           5.20944533e-01, -2.81907786e+00,  7.40148683e-17]),
              'z': array([0., 0., 0., 0., 0., 0., 0.])},
             {'customdata': [transformed_table],
              'line': {'color': 'red'},
              'marker': {'size': 5},
              'mode': 'markers',
              'name': 'Transformed Table',
              'type': 'scatter3d',
              'uid': 'a0ef2588-7b3e-464b-b84f-e8628ae

In [8]:
display(HBox([VBox(WIDGETS), text_widget]))